In [1]:
import os
import pandas as pd 
from datetime import datetime

# Define the absolute path to the data directory (where you want to save the CSV file)
data_dir = 'C:\\Users\\Cristina Jorge\\OneDrive\\Documentos\\GitHub\\Back-End-Elo\\data'


    # Specify the absolute path to the CSV file within the data directory
csv_file_path = os.path.join(data_dir, 'dataELO.csv')
df = pd.read_csv(csv_file_path)
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df.tail(20)

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_11628\1793648326.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')


,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
24059,Série A,2023.0,2023-11-28,Vasco,2.0,4.0,Corinthians,VV,0.0,3.0,351.0
24060,Série A,2023.0,2023-11-29,Bahia,0.0,1.0,São Paulo,VV,0.0,3.0,356.0
24061,Série A,2023.0,2023-11-29,Cuiabá,0.0,2.0,Internacional,VV,0.0,3.0,357.0
24062,Série A,2023.0,2023-11-29,Coritiba,1.0,1.0,Botafogo,E,1.0,1.0,355.0
24063,Série A,2023.0,2023-11-29,Flamengo,0.0,3.0,Atlético-MG,VV,0.0,3.0,352.0
24064,Série A,2023.0,2023-11-29,Palmeiras,4.0,0.0,América-MG,VC,3.0,0.0,353.0
24065,Série A,2023.0,2023-11-29,Santos,0.0,3.0,Fluminense,VV,0.0,3.0,354.0
24066,Série A,2023.0,2023-11-30,Grêmio,2.0,1.0,Goiás,VC,3.0,0.0,358.0
24067,Série A,2023.0,2023-11-30,Red Bull Bragantino,1.0,2.0,Fortaleza,VV,0.0,3.0,359.0
24068,Série A,2023.0,2023-11-30,Cruzeiro,1.0,1.0,Athletico-PR,E,1.0,1.0,360.0


In [2]:
df['GC'] = df['GC'].fillna(0).astype(int)
df['GF'] = df['GF'].fillna(0).astype(int)
df.dtypes

Comp            object
Temp           float64
Data    datetime64[ns]
Casa            object
GC               int32
GF               int32
Fora            object
Res             object
PC             float64
PF             float64
J              float64
dtype: object

In [3]:
#This function updates the Elo ratings and creates a ranking

timestamp_df = pd.DataFrame(columns=['Timestamp'])
def update_elo(df):
    #Gives different weights to each competition
    k_factor_map = {
                    'Série C': 10, 
                    'Série B': 20, 
                    'CBR': 25, 
                    'Série A': 30, 
                    }
    teams = {}
    elo_ranking = {}
    average_elo = {}
    team_games = {}
    team_victories = {}
    team_losses = {}
    team_goals = {}
    team_draws = {}
    last_competition = {}
    current_year = datetime.now().year
    
    #Creates a list of teams and gives then an initial rating of 1500
    for index, row in df.iterrows():
        home_team, away_team = row['Casa'], row['Fora']
        comp_level = row['Comp']
        match_date = row['Data'].year
        k_factor = k_factor_map.get(comp_level, 50) 
        if home_team not in teams:
            teams[home_team] = 1500
            elo_ranking[home_team] = teams[home_team]
        if away_team not in teams:
            teams[away_team] = 1500
            elo_ranking[away_team] = teams[away_team]
        home_elo, away_elo = teams[home_team], teams[away_team]
        home_score, away_score = row['GC'], row['GF']
        result = row['Res']
        # Only update the last competition if it's not 'CBR' and the match is in the current year
        if comp_level != 'CBR' and match_date == current_year:
            last_competition[home_team] = comp_level
            last_competition[away_team] = comp_level

        #Gives different weights for home and away games
        if result == 'VC':
            home_result, away_result = 1, 0
        elif result == 'VV':
            home_result, away_result = 0, 1
        else:
            home_result, away_result = 0.5, 0.5
        home_expected = 1 / (1 + 10 ** ((away_elo - home_elo - 100) / 400))
        away_expected = 1 / (1 + 10 ** ((home_elo - away_elo + 100) / 400))
        
        # Calculates the number of goals
        
        if home_team in team_goals:
            team_goals[home_team] += home_score
        else:
            team_goals[home_team] = home_score

        if away_team in team_goals:
            team_goals[away_team] += away_score
        else:
            team_goals[away_team] = away_score

            team_goals[away_team] = away_score
       # Update the number of victories, losses, and draws for each team
        if result == 'VC':
            home_result, away_result = 1, 0
            if home_team in team_victories:
                team_victories[home_team] += 1
            else:
                team_victories[home_team] = 1
            if away_team in team_losses:
                team_losses[away_team] += 1
            else:
                team_losses[away_team] = 1
        elif result == 'VV':
            home_result, away_result = 0, 1
            if home_team in team_losses:
                team_losses[home_team] += 1
            else:
                team_losses[home_team] = 1
            if away_team in team_victories:
                team_victories[away_team] += 1
            else:
                team_victories[away_team] = 1
        else:
            home_result, away_result = 0.5, 0.5
            if home_team in team_draws:
                team_draws[home_team] += 1
            else:
                team_draws[home_team] = 1
            if away_team in team_draws:
                team_draws[away_team] += 1
            else:
                team_draws[away_team] = 1
        # Update the number of games played for each team
        if home_team in team_games:
            team_games[home_team] += 1
        else:
            team_games[home_team] = 1
        if away_team in team_games:
            team_games[away_team] += 1
        else:
            team_games[away_team] = 1
        
        # Calculate the goal difference and adjust the K factor accordingly
        goal_diff = abs(home_score - away_score)
        if goal_diff >= 4:
            k_factor = k_factor + 0.75 + (goal_diff - 4) / 8
        elif goal_diff == 3:
            k_factor = k_factor + 0.75
        elif goal_diff == 2:
            k_factor = k_factor + 0.5
        else:
            k_factor = k_factor
            
        # Calculate the rating difference factor
        rating_diff = abs(home_elo - away_elo)
        if rating_diff > 400:
            rating_diff = 400
        rating_factor = rating_diff / 400 + 1
            
        teams[home_team] = home_elo + rating_factor * k_factor * (home_result - home_expected)
        teams[away_team] = away_elo + rating_factor * k_factor * (away_result - away_expected)
        teams[home_team] = round(teams[home_team], 2)
        teams[away_team] = round(teams[away_team], 2)
        elo_ranking[home_team] = teams[home_team]
        elo_ranking[away_team] = teams[away_team]
        df.at[index, 'Elo Casa'] = home_elo
        df.at[index, 'Elo Fora'] = away_elo
        df.at[index, 'Novo Elo Casa'] = teams[home_team]
        df.at[index, 'Novo Elo Fora'] = teams[away_team]

        
        for team, elo in teams.items():
            if team in average_elo:
                average_elo[team].append(elo)
            else:
                average_elo[team] = [elo]

    # Compute average Elo for each team
    for team, elo_list in average_elo.items():
        average_elo[team] = round(sum(elo_list) / len(elo_list), 2)
        
  

    #Createa the Datasets
    elo_df = pd.DataFrame.from_dict(elo_ranking, orient='index', columns=['Elo'])
    elo_df = elo_df.sort_values(by='Elo', ascending=False)
    elo_df.index.name = 'Time'
    elo_df.reset_index(inplace=True)
    #elo_df = elo_df[~elo_df['Time'].str.contains(r'\(ARG\)|\(BOL\)|\(CHI\)|\(COL\)|\(EQU\)|\(MEX\)|\(PAR\)|\(PER\)|\(URU\)|\(VEN\)', regex=True)]
    elo_df['Jogos'] = elo_df['Time'].map(team_games) 
    elo_df = elo_df[elo_df['Jogos'] >= 30]
    elo_df['Rank'] = elo_df['Elo'].rank(ascending=False, method='dense')
    elo_df['Média'] = elo_df['Time'].map(average_elo).round(2)
    elo_df['#'] = elo_df['Média'].rank(ascending=False, method='dense')
    elo_df['Gols'] = elo_df['Time'].map(team_goals)
    elo_df['Vitórias'] = elo_df['Time'].map(team_victories)
    elo_df['Derrotas'] = elo_df['Time'].map(team_losses)
    elo_df['Empates'] = elo_df['Time'].map(team_draws)
    elo_df['Série'] = elo_df.apply(lambda row: last_competition.get(row['Time'], '-'), axis=1) 
    elo_df.replace({'Série A': 'A','Série B': 'B','Série C': 'C',                   
            }, inplace=True)
    timestamp_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    timestamp_df.loc[len(timestamp_df)] = [timestamp_now]

    
    return df, elo_df

df, elo_df = update_elo(df)



In [4]:
# Create the data directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

# File names
csv_file_name = 'dataELOranking.csv'
json_file_name = 'dataELOranking.json'
timestamp_json_file_name = 'timestamps.json'

# Construct the absolute paths
csv_file_path = os.path.join(data_dir, csv_file_name)
json_file_path = os.path.join(data_dir, json_file_name)
timestamp_json_file_path = os.path.join(data_dir, timestamp_json_file_name)

# Save the cleaned DataFrame to CSV
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# Save to JSON files
elo_df.to_json(json_file_path, orient='records')
timestamp_df.to_json(timestamp_json_file_path)

# Display the cleaned DataFrame
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Elo Casa,Elo Fora,Novo Elo Casa,Novo Elo Fora
0,CBR,2003.0,2003-02-05,CFZ-DF,0,0,Fortaleza,E,1.0,1.0,11.0,1500.00,1500.00,1496.50,1503.50
1,CBR,2003.0,2003-02-05,Bangu,2,1,Gama,VC,3.0,0.0,12.0,1500.00,1500.00,1509.00,1491.00
2,CBR,2003.0,2003-02-05,Pelotas,1,2,Guarani,VV,0.0,3.0,10.0,1500.00,1500.00,1484.00,1516.00
3,CBR,2003.0,2003-02-05,Botafogo-PB,1,4,Flamengo,VV,0.0,3.0,7.0,1500.00,1500.00,1483.52,1516.48
4,CBR,2003.0,2003-02-05,Itabaiana,0,1,Vasco,VV,0.0,3.0,8.0,1500.00,1500.00,1484.00,1516.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24074,Série A,2023.0,2023-12-03,Botafogo,0,0,Cruzeiro,E,1.0,1.0,365.0,1780.21,1718.40,1772.68,1725.93
24075,Série A,2023.0,2023-12-03,Palmeiras,1,0,Fluminense,VC,3.0,0.0,366.0,1872.28,1810.45,1882.07,1800.66
24076,Série A,2023.0,2023-12-03,Red Bull Bragantino,1,0,Coritiba,VC,3.0,0.0,367.0,1781.80,1643.65,1789.97,1635.48
24077,Série A,2023.0,2023-12-03,Grêmio,1,0,Vasco,VC,3.0,0.0,368.0,1806.33,1732.60,1815.88,1723.05


In [5]:
df.tail(30)

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Elo Casa,Elo Fora,Novo Elo Casa,Novo Elo Fora
24049,Série B,2023.0,2023-11-25,Ponte Preta,3,0,CRB,VC,3.0,0.0,378.0,1539.61,1634.77,1552.27,1622.11
24050,Série B,2023.0,2023-11-25,Ceará,1,3,Juventude,VV,0.0,3.0,371.0,1612.13,1628.44,1598.94,1641.63
24051,Série B,2023.0,2023-11-25,Botafogo-SP,0,1,Londrina,VV,0.0,3.0,379.0,1542.66,1469.37,1525.37,1486.66
24052,Série A,2023.0,2023-11-26,Atlético-MG,3,0,Grêmio,VC,3.0,0.0,349.0,1834.32,1808.75,1845.01,1798.06
24053,Série A,2023.0,2023-11-26,Botafogo,1,1,Santos,E,1.0,1.0,348.0,1788.19,1761.79,1782.62,1767.36
24054,Série A,2023.0,2023-11-26,São Paulo,0,0,Cuiabá,E,1.0,1.0,347.0,1779.91,1768.55,1775.13,1773.33
24055,Série A,2023.0,2023-11-26,Fortaleza,2,2,Palmeiras,E,1.0,1.0,344.0,1722.79,1869.48,1725.53,1866.74
24056,Série A,2023.0,2023-11-26,América-MG,0,3,Flamengo,VV,0.0,3.0,345.0,1614.71,1870.38,1600.10,1884.99
24057,Série A,2023.0,2023-11-26,Internacional,1,0,Red Bull Bragantino,VC,3.0,0.0,346.0,1745.51,1825.65,1762.49,1808.67
24058,Série A,2023.0,2023-11-27,Goiás,0,1,Cruzeiro,VV,0.0,3.0,350.0,1683.46,1701.30,1664.15,1720.61


In [6]:
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Elo Casa,Elo Fora,Novo Elo Casa,Novo Elo Fora
0,CBR,2003.0,2003-02-05,CFZ-DF,0,0,Fortaleza,E,1.0,1.0,11.0,1500.00,1500.00,1496.50,1503.50
1,CBR,2003.0,2003-02-05,Bangu,2,1,Gama,VC,3.0,0.0,12.0,1500.00,1500.00,1509.00,1491.00
2,CBR,2003.0,2003-02-05,Pelotas,1,2,Guarani,VV,0.0,3.0,10.0,1500.00,1500.00,1484.00,1516.00
3,CBR,2003.0,2003-02-05,Botafogo-PB,1,4,Flamengo,VV,0.0,3.0,7.0,1500.00,1500.00,1483.52,1516.48
4,CBR,2003.0,2003-02-05,Itabaiana,0,1,Vasco,VV,0.0,3.0,8.0,1500.00,1500.00,1484.00,1516.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24074,Série A,2023.0,2023-12-03,Botafogo,0,0,Cruzeiro,E,1.0,1.0,365.0,1780.21,1718.40,1772.68,1725.93
24075,Série A,2023.0,2023-12-03,Palmeiras,1,0,Fluminense,VC,3.0,0.0,366.0,1872.28,1810.45,1882.07,1800.66
24076,Série A,2023.0,2023-12-03,Red Bull Bragantino,1,0,Coritiba,VC,3.0,0.0,367.0,1781.80,1643.65,1789.97,1635.48
24077,Série A,2023.0,2023-12-03,Grêmio,1,0,Vasco,VC,3.0,0.0,368.0,1806.33,1732.60,1815.88,1723.05


In [8]:
elo_df.head(20)

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
0,Palmeiras,1882.07,957,1.0,1722.88,2.0,1406,438,247,272,A
1,Atlético-MG,1877.94,980,2.0,1701.99,7.0,1421,411,291,278,A
2,Flamengo,1870.43,993,3.0,1721.85,3.0,1379,431,263,299,A
3,Grêmio,1815.88,959,4.0,1706.55,5.0,1267,407,274,278,A
4,Internacional,1803.25,951,5.0,1705.18,6.0,1239,407,271,273,A
5,Fluminense,1800.66,987,6.0,1686.27,9.0,1316,386,314,287,A
6,Red Bull Bragantino,1789.97,645,7.0,1572.81,27.0,831,249,231,165,A
7,Athletico-PR,1788.07,882,8.0,1668.84,11.0,1167,351,309,222,A
8,São Paulo,1784.90,942,9.0,1727.05,1.0,1296,406,246,290,A
9,Corinthians,1774.67,971,10.0,1711.20,4.0,1231,399,261,311,A
